In [1]:
#;.pykx.disableJupyter()

In [2]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


<img src="../qbies.png" width="50px" style="width: 100px;padding-right:5px;padding-top:1px;padding-left:5px;" align="left"/>

   # Dictionaries Practical Guidance

# Dictionary lookups 
Associations and dictionaries are useful structures for storing lookup, static or rule type information. 

For example, let’s say we have a list of incoming bids and we wish to implement the following simple rule-if the first bid is null, replace this null with 0.0.

Below is a list of incoming bids where the first value is null.

In [ ]:
bids:0n 1 2 3f
if[null first bids;bids[0]:0.0]  /if initial bid is null, replace with 0
bids

Associations provide an easy way to extend this solution to handle more than one datatype. For example, the following association provides initial values for simple lists of type char, symbol, int and float.

In [ ]:
show iv:10 11 6 9h!("f";`first;0i;0.0) //default values for 4 datatypes

Assume the variable `data` is the list of incoming data. At runtime,`data` will be of one of these four datatypes.The null replacement expression becomes:

In [ ]:
data:``a`c`h /data is a list of symbols
if[null first data;data[0]:iv type data]
data

This association is easily extendable to all types.

This same idea can also be used in other situations. Suppose we have a list of error messages in code format. The association below explains the code:

In [ ]:
code:`c`t`o!`CorruptData`TypeMismatch`OutofRange

If we have a list of errors that occurred within a program, it might be useful to decode this so that it is easily readable to a person reviewing the list.

In [ ]:
errors:`c`o`t`o`o`c`t`t`o`c`c
code[errors]

# Dictionaries & Code readability

When dictionaries get large they can become very difficult to read in our code - we can keep our dictionaries "clear" by using iterators.

In [3]:
`is`this`the`real`world`or`is`this`just`fantasy!(("caught";"in a landslide");`no`escape`from;`reality;2 3 ;(2;2f;2);.z.d;`the`neverending;`story;1;"!!!!")

is     | ("caught";"in a landslide")
this   | `no`escape`from
the    | `reality
real   | 2 3
world  | (2;2f;2)
or     | 2025.02.25
is     | `the`neverending
this   | `story
just   | 1
fantasy| "!!!!"


Dictionaries can be generated in a more clear and readable format if we use some tricks - like creating  key-values as pairs first, and then flipping them into two separate lists. We can then use the `.` operator to make it clear we want the lists to be considered the first and second arguments to the `!` operator.

In [4]:
.[!; flip (
            (`is;("caught";"in a landslide"));
            (`this;`no`escape`from);
            (`the;`reality);
            (`real;2 3);
            (`world;(2;2f;2));
            (`or;.z.d);
            (`is;`the`neverending);
            (`this;`story);
            (`just;1);
            (`fantasy;"!!!!")
          )]

is     | ("caught";"in a landslide")
this   | `no`escape`from
the    | `reality
real   | 2 3
world  | (2;2f;2)
or     | 2025.02.25
is     | `the`neverending
this   | `story
just   | 1
fantasy| "!!!!"


If we move our "make these lists a dictionary" operation into a projection of `.` with `!` i.e. `.[!]`, we can avoid having to wrap our whole dictionary lists. In this situation, it is easier to see what dictionary we are trying to generate within the code, i.e without having to output to the console.

In [5]:
keyVals:    ((`is;("caught";"in a landslide"));
            (`this;`no`escape`from);
            (`the;`reality);
            (`real;2 3);
            (`world;(2;2f;2));
            (`or;.z.d);
            (`is;`the`neverending);
            (`this;`story);
            (`just;1);
            (`fantasy;"!!!!"))
flip keyVals         //our list of (keys;values)

//"make these lists a dictionary"
.[!] flip keyVals

is                          this            the      real world    or        ..
("caught";"in a landslide") `no`escape`from `reality 2 3  (2;2f;2) 2025.02.25..
is     | ("caught";"in a landslide")
this   | `no`escape`from
the    | `reality
real   | 2 3
world  | (2;2f;2)
or     | 2025.02.25
is     | `the`neverending
this   | `story
just   | 1
fantasy| "!!!!"


# Dictionaries as Function Input 

It's good practice to use dictionaries as input to functions when: 
+ The parameters might change (e.g. two inputs atm, but might need to add more, or remove some) - less updating required in the code base as the function's valence doesn't change.
+ The parameters are variable in nature (e.g. sometimes three, sometimes four) - avoid rank errors and unexpected projections 
+ We want to set default parameter values - can set default parameters more easily using dictionaries 
+ We have a large number of parameters, exceeding eight even - can't have more than 8 explicit arguments in functions so this avoids that. 
+ We want improved code readability - it's usually easier to read code where we reference inputs like <code>param[\`capsSetting]</code> versus `param[2]`.

Worked example of a function which checks whether a student passed or not. The function takes a parameter dictionary as input and returns a string which mentions the result.

In [ ]:
checkPassed:{[paramDict]
    //check required parameters 
    requiredParams: `name`result;
    if[not all requiredParams in key paramDict;
            -2 "Error: missing required values:","," sv string requiredParams except key paramDict; 
            :0b];  //force returning a value of 0b from the function to indicate function failure
    //other input checks could be added - e.g. that name and result are of the right format 
            
    //setting default values
    defaultParams:.[!] flip ((`passLevel;70);
                            (`otherParams;`etc));
    
    //using dictionary coalescing to combine the passed parameters and our default inputs 
    params:defaultParams^paramDict;

    //checking if the result is above the pass mark
    res:params[`result] > params[`passLevel];
    //using this result to index into a condition list 
    pass:$[res;" did "; " did not "],"pass,";

    //returning a string indication of whether they passed or failed 
    "The participant ",string[params[`name]],pass," with a result of:",string[params[`result]]
    
 }

There is a lot happening in the above function so read through carefully and make sure you understand what each line is doing.

Now we have the function defined we can use it to check some results: 

In [ ]:
//passing an empty dictionary alarms that we don't have the right input 
checkPassed[()!()]           

//passing a dictionary with just the required input
checkPassed[`name`result!(`rebecca;50)] 

//passing a dictionary with the required input and an optional parameter - changing the passLevel param
checkPassed[`name`result`passLevel!(`rebecca;50;30)]  

# Why we shouldn't use dot notation 
In our q-bee note we advise against using dot notation to update or retrieve values from dictionaries - heres why! 

Whenever we use dot (`.`) notation we need the dictionary to be both untyped, and global. In general in kdb+/q, any dot notation used is referencing a global value and so will error if this isn't found globally, or in some cases we might be insidiously updating values that we didn't intend.

Here's an example of a case where using dot notation won't work: 

In [6]:
show d: `a`b`c! 1 2 3;   //not an untyped dictionary
/ d.b                      //errors
d.this:2                 //also errors - comment the line above to see

a| 1
b| 2
c| 3


QError: d.this

In order for this syntax to work, we change our dictionary to be untyped, and this will now work globally:  

In [7]:
show d: ``a`b`c! (::;1; 2 ;3) ;  //untyped dictionary
d.b
d.this:2
d

2
    | ::
a   | 1
b   | 2
c   | 3
this| 2
 | ::
a| 1
b| 2
c| 3


This however can lead to some odd behaviour within a function that might not have been intended by the creator: 

In [ ]:
//defining a value of d globally
show d: ``a`b`c! (::;1; 2 ;3) ;


{ d: ``a`b`c! (::;6; 6 ;6);
    -1 "Dictionary definition within function:"; 
    show d; 
    -1 "Dictionary value for `b returned when using dot notation:",string d.b;
    //referencing the global value!! 
    d.b }[]

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> <b>All of these problems can be avoided!!</b> If we use functional notation (aka indexing notation) all of the above will return the value we expect. We can think of the key values of a dictionary as the indexes in this context.  </i></p>

In [ ]:
show d: `a`b`c! 1 2 3;   //not an untyped dictionary
show d[`b]                   
show d[`this]:2    

In [ ]:
show d: ``a`b`c! (::;1; 2 ;3) ;  //untyped dictionary
show d[`b]
show d[`this]:2
show d   

In [ ]:
//defining a value of d globally
show d: ``a`b`c! (::;1; 2 ;3) ;


{ d: ``a`b`c! (::;6; 6 ;6);
    -1 "Dictionary definition within function:"; 
    show d; 
    -1 "Dictionary value returned when using functional notation:",string d[`b];
    //referencing the within function scope value!! 
    d[`b] }[]

This doesn't mean you should *never* use dot notation, it's just not best practice. If there is a case in which you need to use it just use with care and remember it references global values and ignores function scope! 

# Dictionaries and Tables - the link, column dictionaries 
Dictionaries and tables are intrinsically related - in fact a table is just a list of dictionaries! 

##### Column dictionaries
Column dictionaries are the foundation for tables. They are a special subset of dictionaries where the mapping is from a simple list of symbols to a rectangular list of lists. We can create a table from dictionaries of this format. 

In [ ]:
d: `col1`col2`col3!(1 2 2; 7 5 2; `a`b`cc)
flip d

In [ ]:
d: string[`col1`col2`col3]!(1 2 2; 7 5 2; `a`b`cc)  //not a symbol list key
flip d                                              //therefore can't be interpreted as a table

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> We can only <code>flip</code> rectangular dictionaries - i.e. dictionaries where each key  is assigned to lists of the same length.</i></p>
 

Similarly we can `flip` a table to return to a dictionary:

In [8]:
show t:([]name:`john`steve`rachel;ages:20 31 22) //defining a table
flip t

name| john steve rachel
ages| 20   31    22    
name   ages
-----------
john   20  
steve  31  
rachel 22  


We could also have defined the above table from a dictionary as follows: 

In [9]:
t2: `name`ages!(`john`steve`rachel;20 31 22)
flip t2
t~flip t2   //are these the same?

name   ages
-----------
john   20  
steve  31  
rachel 22  
1b


<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Each row in a kdb+/q table is actually a dictionary! This is pretty central to understanding the relationship between tables and dictionaries.</i></p>
 

We can prove this by indexing into a row of our table: 

In [10]:
t[0]     //first row of our table

name| `john
ages| 20


##### Atomic dictionaries  -> Tables
A nuance to this, is that tables in kdb+/q require their columns to be lists so if we have an atomic value dictionary we can't `flip` this to a table in the same way: 

In [11]:
d: `a`b!1 2
flip d

QError: rank

If we remember that a table really is a list of dictionaries, we can create a table from a list of dictionaries (with conforming keys): 

In [13]:
/ (d;d)     //simple table
enlist d  //therefore, enlist our atomic dictionary will create a table!

a b
---
1 2


# Namespaces

[`Namespaces`](https://code.kx.com/q4m3/12_Workspace_Organization/#120-overview) is the idea of placing a (usually hierarchical) structure on names. Namespacing in q is implemented with dictionaries. A context is a dictionary whose keys are the symbolic names of the variables in the corresponding namespace. The context dictionary associates each variable name with its currently assigned value.

A context is a specially formatted q dictionary. Its keys are the symbolic names of the variables it contains; each is associated with the current value of the corresponding variable.

The context dictionaries are dictionaries that live in the workspace along with ordinary variables, which have special meaning to q. We can perform operations on them just as with our own dictionaries.

When we start a fresh q session, the root dictionary is empty. As with any variable, we can reveal the root context by applying `get` (or `value`) to its name and then applying the utility [`.Q.s1`](https://code.kx.com/q/ref/dotq/#qs1-string-representation) to display its internal form since it is empty.

Let's restart our Kernel before doing this.

In [14]:
.Q.s1  get `.

`keyVals`d`t`t2!(((`is;("caught";"in a landslide"));(`this;`no`escape`from);`..


Here we can see all the symbols and their values in the current root namespace.

Let's define a variable and a function

In [15]:
a:5
func : {x+y}

In [16]:
get `.

keyVals| ((`is;("caught";"in a landslide"));(`this;`no`escape`from);`the`real..
d      | `a`b!1 2
t      | +`name`ages!(`john`steve`rachel;20 31 22)
t2     | `name`ages!(`john`steve`rachel;20 31 22)
a      | 5
func   | {x+y}


The entries in a context dictionary are sorted by name

Things get more interesting when we use namespacing and create a context.

In [17]:
.foo.bar : 50

get `.
get `.foo

keyVals| ((`is;("caught";"in a landslide"));(`this;`no`escape`from);`the`real..
d      | `a`b!1 2
t      | +`name`ages!(`john`steve`rachel;20 31 22)
t2     | `name`ages!(`john`steve`rachel;20 31 22)
a      | 5
func   | {x+y}
   | ::
bar| 50


Observe that the newly created context dictionary for .foo is not physically in the root directory; it is a separate dictionary.

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> The significance of the :: in the first entry of the .foo context dictionary is to prevent the value list of the context dictionary from collapsing to a simple list in case all the variables should have values of the same scalar type. This would then prevent subsequent definition of variables of other types in that context.</i></p>
 


# Advanced - Dictionaries and attributes
[Attributes](https://code.kx.com/q4m3/8_Tables/#88-attributes) are metadata (e.g. like tags) in kdb+/q that indicate that the referenced data is in a particular format. When we apply attributes to dictionaries we can achieve some interesting results

## Sorted attribute - Step function 

When we apply a sorted attribute to our dictionary we can use it as a [step function](https://en.wikipedia.org/wiki/Step_function) - namely we can retrieve values from our dictionary that don't have explicit keys. 

For example, imagine we wanted to know the associated [viscosity of water depending on its temperature](https://www.engineersedge.com/physics/water__density_viscosity_specific_weight_13146.htm) - we can encode that into a dictionary: 

In [18]:
show tempVisc: 10 20 30 40 50f! 1.308 1.002 0.7978 0.6531 0.5471
tempVisc[10f]
tempVisc[15f]

1.308
0n
10| 1.308
20| 1.002
30| 0.7978
40| 0.6531
50| 0.5471


If we apply a sorted attribute to our dictionary, it will becomes a step function:

In [19]:
show step: `s#tempVisc
step[10f]
step[15 35 29.99999f]   //returns the value associated with the largest key that our input exceeds
                        //aka - it returns the previous "step"

1.308
1.308 0.7978 1.002
10| 1.308
20| 1.002
30| 0.7978
40| 0.6531
50| 0.5471


## Unique attribute - consistent performance
When you know that the keys are unique you can apply the <code>\`u#</code> attribute to the keys. This will effectively cause the dictionary to be a hash table with the attendant improvement in lookup speed over the default linear lookup.

In [20]:
(`u#`a`b`c)!10 20 30

a| 10
b| 20
c| 30
